# Load Validation Example 
Security Note: No PII, PHI, or Company identifiable information, and data structures have been masked. 
 
### Purpose: 
An essential query to validate across databases monthly target payment amounts. This was done in both dev and production environments. Total magnitude of monthly payment was around $35M. 
 
### Key Concepts
* Parameters
* Temp Table operations
* Left Joins
* Utilization of null values
* Case statements
* Sub-Queries 
* Unions 
 
### Walkthrough:
I’ll keep the walkthrough brief in the interest of time, and this is the shorter script of my examples. I am the sole author of this script and wrote it from scratch. 

I’ll start with the inner-most query and work our way out. The inner-most subquery contains union-ed summations for each of the 8 types of payment amounts for the current process month. This is the core of the validation. 
 
An outer-query consisting of two case statements was needed to establish a consistent order the table results would be returned in and a variance between expected target parameters and the amounts currently in the database. I aliased the inner and outer query together simply as b. 
Expanding further out, a temp table was created to determine the presence of the payment table in the database for that month. Some months it was normal and expected to not have data in certain payment tables. The temp table had hard-coded table names so that when it was left joined to query b, any empty payment tables would clearly appear as null in the final output. 
 
In summary, this was a critical validation query I authored that utilized a variety of SQL concepts. 

In [ ]:
DECLARE @MED_CLAIM_TARGET NUMERIC(18,2) =  XXXXXXX.XX  
DECLARE @OSA_CLAIM_TARGET NUMERIC(18,2) = 0
DECLARE @OSA_TARGET NUMERIC(18,2) = 0
DECLARE @RX_CLAIM_TARGET NUMERIC(18,2) = XXXXXXX.XX 
DECLARE @OBH_CLAIM_TARGET NUMERIC(18,2) = XXXXXXX.XX 
DECLARE @CAP_PAYMENT_TARGET NUMERIC(18,2) = XXXXXXXX.XX 
DECLARE @CAP_ADJUSTMENT_TARGET NUMERIC(18,2) = -XXXXX.XX 
DECLARE @POOLING_TARGET NUMERIC(18,2) =  -XXXXXX.XX


--Temp created to have hard-coded fixed table values compare to queried tables. Tbl's with no data will appear NULL when left joined. 
IF object_id('tempdb..##TABLE_VALIDATION') IS NOT NULL 
DROP TABLE [##TABLE_VALIDATION]
;
CREATE TABLE [tempdb].[##TABLE_VALIDATION] (
	[SORT_ORDER] [int] NULL,
	[TBL] [varchar](50) NULL
)
;

INSERT INTO [tempdb].[##TABLE_VALIDATION] (SORT_ORDER, TBL)
VALUES
	
	(1 , 'TABLE_1'),
	(2 , 'TABLE_2'),
	(3 , 'TABLE_3'),
	(4 , 'TABLE_4'),
	(5 , 'TABLE_5'),
	(6 , 'TABLE_6'),
	(7 , 'TABLE_7'),
	(8 , 'TABLE_8'),
	(9 , 'TABLE_9')
;

SELECT 
a.TBL,
b.TBL_QUERIED, b.PROCESS_MONTH, b.FLIP_DATE, b.Total_load_database_Amount, b.VARIANCE

FROM [tempdb].[##TABLE_VALIDATION] AS a
LEFT JOIN
(
	SELECT 
	--Outer query establishes SORT_ORDER and VARIANCE from expected targets for only returned tables. 
		CASE 
			WHEN TBL_QUERIED = 'TABLE_1' THEN 1
			WHEN TBL_QUERIED = 'TABLE_2' THEN 2
			WHEN TBL_QUERIED = 'TABLE_3' THEN 3
			WHEN TBL_QUERIED = 'TABLE_4' THEN 4
			WHEN TBL_QUERIED = 'TABLE_5' THEN 5
			WHEN TBL_QUERIED = 'TABLE_6' THEN 6
			WHEN TBL_QUERIED = 'TABLE_7' THEN 7
			WHEN TBL_QUERIED = 'TABLE_8' THEN 8
			WHEN TBL_QUERIED = 'TABLE_9' THEN 9
		END AS SORT_ORDER, 

		TBL_QUERIED, PROCESS_MONTH, FLIP_DATE, Total_load_database_Amount,

		CASE 
			WHEN TBL_QUERIED = 'TABLE_1' THEN (Total_load_database_Amount - @MED_CLAIM_TARGET)
			WHEN TBL_QUERIED = 'TABLE_2' THEN (Total_load_database_Amount - @OSA_CLAIM_TARGET)
			WHEN TBL_QUERIED = 'TABLE_3' THEN (Total_load_database_Amount - @OSA_TARGET)
			WHEN TBL_QUERIED = 'TABLE_4' THEN (Total_load_database_Amount - @RX_CLAIM_TARGET)
			WHEN TBL_QUERIED = 'TABLE_5' THEN (Total_load_database_Amount - @OBH_CLAIM_TARGET)
			WHEN TBL_QUERIED = 'TABLE_6' THEN (Total_load_database_Amount - @CAP_PAYMENT_TARGET)
			WHEN TBL_QUERIED = 'TABLE_7' THEN (Total_load_database_Amount - @CAP_ADJUSTMENT_TARGET)
			WHEN TBL_QUERIED = 'TABLE_8' THEN (Total_load_database_Amount - @POOLING_TARGET)
			WHEN TBL_QUERIED = 'TABLE_9' THEN 0.00 --Funding Obligation is not validated against 'XXXX'
		END AS VARIANCE	

	FROM
	-- Inner query contains summations for each of the 8 types of payment amounts and funding obligation for current process month. 
			(
			SELECT 'TABLE_1' AS TBL_QUERIED,  
			CAST(CONCAT(ACCOUNTING_PERIOD_ID,'01') AS DATE) AS PROCESS_MONTH, FLIP_DATE,
			SUM(PAID_AMOUNT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_1]
			GROUP BY CAST(CONCAT(ACCOUNTING_PERIOD_ID,'01') AS DATE), FLIP_DATE

			UNION

			SELECT 'TABLE_2' AS TBL_QUERIED, PROCESS_MONTH, FLIP_DATE, SUM(PAID_AMOUNT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_2]
			GROUP BY PROCESS_MONTH, FLIP_DATE

			UNION

			SELECT 'TABLE_3' AS TBL_QUERIED, PROCESS_MONTH, FLIP_DATE, SUM(OSAAmount) AS Total_load_database_Amount
			FROM [dbo].[TABLE_3]
			GROUP BY PROCESS_MONTH, FLIP_DATE

			UNION

			SELECT 'TABLE_4' AS TBL_QUERIED, CAST(CONCAT(PROCESS_MONTH,'01') AS DATE) AS PROCESS_MONTH, FLIP_DATE, 
			SUM(PAID_AMOUNT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_4]
			GROUP BY CAST(CONCAT(PROCESS_MONTH,'01') AS DATE), FLIP_DATE

			UNION

			SELECT 'TABLE_5' AS TBL_QUERIED, PROCESS_MONTH, FLIP_DATE, SUM(PAID_AMOUNT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_5]
			GROUP BY PROCESS_MONTH, FLIP_DATE

			UNION

			SELECT 'TABLE_6' AS TBL_QUERIED, FILEDATE AS PROCESS_MONTH, FLIP_DATE, SUM(PAID_AMOUNT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_6]
			GROUP BY FILEDATE, FLIP_DATE

			UNION

			SELECT 'TABLE_7' AS TBL_QUERIED, FILEDATE AS PROCESS_MONTH, FLIP_DATE, SUM(ALLOCATEDADJUSTMENT) AS Total_load_database_Amount
			FROM [dbo].[TABLE_7]
			GROUP BY FILEDATE, FLIP_DATE

			UNION

			SELECT 'TABLE_8' AS TBL_QUERIED, PROCESS_MONTH, FLIP_DATE, SUM(pooling_amount_member) AS Total_load_database_Amount
			FROM [dbo].[TABLE_8]
			GROUP BY PROCESS_MONTH, FLIP_DATE

			UNION

			SELECT 'TABLE_9' AS TBL_QUERIED, DATEADD(month,-1, FLIP_DATE) AS PROCESS_MONTH, FLIP_DATE, SUM(MAXFUNDINGOBLIGATION) AS Total_load_database_Amount
			FROM [dbo].TABLE_9
			GROUP BY DATEADD(month,-1, FLIP_DATE), FLIP_DATE
		
			) AS AllFileTypes

	) AS b
ON a.SORT_ORDER = b.SORT_ORDER
ORDER BY a.SORT_ORDER